In [1]:
import util;
import numpy as np;
import visualize;
# import sklearn;
import os;
import time;
from sklearn.cluster import KMeans

def getDataAndLabels(data_path,batchSize,numBatches,fc6_dir,dim_size=4096,limit=-1):
    lines=util.readLinesFromFile(data_path);
    if limit>-1:
        lines=lines[:limit]
    out_data=np.zeros((len(lines),dim_size));
#     print out_data.shape,limit,dim_size
    start_curr=0;
    for i in range(numBatches):
        file_curr=os.path.join(fc6_dir,str(i+1)+'.npy');
        fc6_curr=np.load(file_curr);
#         print file_curr;
        end_curr=min(start_curr+fc6_curr.shape[0],len(lines));
        len_curr=end_curr-start_curr
        out_data[start_curr:end_curr,:]=fc6_curr[:len_curr];
        start_curr=end_curr;
    return out_data,lines;

def normalizeData(data):
    norm_data=np.linalg.norm(data,axis=1);
    norm_data=np.expand_dims(norm_data,axis=1);
    norm_data=np.tile(norm_data,(1,data.shape[1]));
    data=data/norm_data;
    return data;

def getClusterIdx(data,num_clusters):
#     norm_data=np.linalg.norm(data,axis=1);
#     norm_data=np.expand_dims(norm_data,axis=1);
#     norm_data=np.tile(norm_data,(1,data.shape[1]));
#     data=data/norm_data;
    data=normalizeData(data)
    kmeaner=KMeans(n_clusters=num_clusters,n_jobs=12);
    cluster_idx=kmeaner.fit_predict(data);
    return cluster_idx;

def saveClusterIdx(data_path,batchSize,numBatches,fc6_dir,num_clusters,out_file_clusters,dim_size=4096,limit=-1):
    data,labels = getDataAndLabels(data_path,batchSize,numBatches,fc6_dir,dim_size,limit);
    cluster_idx = getClusterIdx(data,num_clusters);
    print out_file_clusters,cluster_idx.shape
    np.save(out_file_clusters,cluster_idx);

def makeClusterHTML(out_file_html,labels,num_cols,size_im,dir_server):
    ims=[];
    captions=[];
    start_idx=0;
    while start_idx<len(labels):
        row_curr=[];
        caption_curr=[];
        if start_idx+num_cols>len(labels):
            num_cols_real=len(labels)-start_idx;
        else:
            num_cols_real=num_cols;
        for col_no in range(num_cols_real):
            idx_curr=start_idx+col_no;
            label_curr=labels[idx_curr];
            row_curr.append(util.getRelPath(label_curr,dir_server));
            caption_curr.append('');
        ims.append(row_curr);
        captions.append(caption_curr);
        start_idx=start_idx+num_cols_real;
    visualize.writeHTML(out_file_html,ims,captions,size_im,size_im);
    print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')

def script_makeClusterHTML(labels,clusters,out_dir,dir_server):
    labels=np.array(labels);
    clusters_uni=list(set(clusters));
    labels_clusters=[];
    for cluster_idx in clusters_uni:
        labels_rel=labels[clusters==cluster_idx];
        print len(labels_rel);
        labels_clusters.append(labels_rel);
        out_file_html=os.path.join(out_dir,str(cluster_idx)+'.html');
        num_cols=40;
        size_im=20;
        makeClusterHTML(out_file_html,labels_rel,num_cols,size_im,dir_server);


def getDotProduct(out_file_dot,
                  batchSize,
                  dim_size,
                  limit,
                horse_data_path,
                  horse_numBatches,
                  horse_fc6_dir,
                  human_data_path,
                 human_numBatches,
                 human_fc6_dir):
    horse_data,horse_labels=getDataAndLabels(horse_data_path,batchSize,horse_numBatches,horse_fc6_dir,dim_size,limit);
    horse_data=normalizeData(horse_data);
    
    face_data,face_labels=getDataAndLabels(human_data_path,batchSize,human_numBatches,human_fc6_dir,dim_size,limit);
    face_data=normalizeData(face_data);
#     print (horse_data.shape,face_data.shape);
    dot_product=np.dot(horse_data,face_data.T);
#     print (dot_product.shape);

    np.save(out_file_dot,dot_product)


In [ ]:
def main():
    print 'hello';
    dir_server='/home/SSD3/maheen-data';
    
    data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/pairs_all.txt';
    out_dir='/home/SSD3/maheen-data/horse_project/sanityCheckHorse_face'
    fc6_dir=os.path.join(out_dir,'horse_fc6');
    numBatches=60;
    
#     data_path='/home/SSD3/maheen-data/data_face_network/aflw_cvpr_train.txt';
#     out_dir='/home/SSD3/maheen-data/horse_project/sanityCheckHorse_face'
#     fc6_dir=os.path.join(out_dir,'face_fc6');
#     numBatches=493;
    
    batchSize=64;
    

    num_clusters=32;
    dim_size=100;
    limit=-1;
    out_file_clusters=os.path.join(fc6_dir,'clusters.npy');
    saveClusterIdx(data_path,batchSize,numBatches,fc6_dir,num_clusters,out_file_clusters,dim_size,limit);

    labels=util.readLinesFromFile(data_path);
    labels=[line_curr[:line_curr.index(' ')] for line_curr in labels];
    if limit>-1:
        labels=labels[:limit];

    clusters=np.load(out_file_clusters);
    script_makeClusterHTML(labels,clusters,out_dir,dir_server)

    data,labels=getDataAndLabels(data_path,batchSize,numBatches,fc6_dir,dim_size,limit);
    data=normalizeData(data);
    clusters_uni=list(set(clusters));
    for cluster_idx in clusters_uni:
        data_rel=data[clusters==cluster_idx,:];
        labels_rel=data[clusters==cluster_idx];
        pairwise=np.dot(data_rel,data_rel.T);
        print pairwise.shape;
        print np.min(pairwise),np.max(pairwise);
        break;

main();

In [ ]:

def getImgPaths(dot_product,top_num,bottom_num,horse_paths,human_paths,dir_server):
    img_paths=[];
    idx_sort=np.argsort(dot_product,1)[:,::-1];
    assert dot_product.shape[0]==len(horse_paths);
    assert dot_product.shape[1]==len(human_paths);
    for idx in range(dot_product.shape[0]):
        dot_sorted=dot_product[idx,idx_sort[idx]];   
        horse_curr=horse_paths[idx];

        if top_num>0:
            human_best=[human_paths[idx_curr] for idx_curr in idx_sort[idx,:top_num]];
            best_row=[horse_curr]+human_best;
            best_row=[util.getRelPath(file_curr,dir_server) for file_curr in best_row]
            img_paths.append(best_row);
            
        if bottom_num>0:
            human_worst=[human_paths[idx_curr] for idx_curr in idx_sort[idx,-bottom_num:]];
            worst_row=[horse_curr]+human_worst;
            worst_row=[util.getRelPath(file_curr,dir_server) for file_curr in worst_row]
            img_paths.append(worst_row);
    return img_paths;

def main():
    print 'hello';
    dir_server='/home/SSD3/maheen-data';
    click_str='http://vision1.idav.ucdavis.edu:1000'
    batchSize=64;
    num_clusters=32;
    dim_size=100;
    limit=-1;
    out_dir='/home/SSD3/maheen-data/horse_project/sanityCheckHorse_face'
    out_file_dot=os.path.join(out_dir,'dot_product.npy')
    out_file_html=os.path.join(out_dir,'matches.html');
    top_num=5;
    bottom_num=0;
    
    horse_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/pairs_all.txt';
    horse_fc6_dir=os.path.join(out_dir,'horse_fc6');
    horse_numBatches=60;
    

    human_data_path='/home/SSD3/maheen-data/data_face_network/aflw_cvpr_train.txt';
    human_fc6_dir=os.path.join(out_dir,'face_fc6');
    human_numBatches=493;
    
#     getDotProduct(out_file_dot,batchSize,dim_size,limit,\
#                 horse_data_path,horse_numBatches,horse_fc6_dir,\
#                   human_data_path,human_numBatches,human_fc6_dir);
    
    dot_product=np.load(out_file_dot);
    data_horse=util.readLinesFromFile(horse_data_path);
    data_horse=[file_curr.split(' ')[0] for file_curr in data_horse]
    data_human=util.readLinesFromFile(human_data_path);
    data_human=[file_curr.split(' ')[0] for file_curr in data_human]
    
    img_paths=getImgPaths(dot_product,top_num,bottom_num,data_horse,data_human,dir_server);
    captions=[];

    for r in range(len(img_paths)):
        caption_row=[];
        for c in range(len(img_paths[r])):
            caption_row.append(' ');
        captions.append(caption_row);
    
    visualize.writeHTML(out_file_html,img_paths,captions,100,100);
    print out_file_html.replace(dir_server,click_str)
#     dot_curr=dot_product[idx];
   
#     print idx_sort.shape
#     horse_curr=data_horse[idx];
    
    
    
    print 'done'

main();

In [10]:
def getImgPaths(dot_product,top_num,bottom_num,horse_paths,human_paths,dir_server):
    img_paths=[];
    idx_sort=np.argsort(dot_product,1)[:,::-1];
    assert dot_product.shape[0]==len(horse_paths);
    assert dot_product.shape[1]==len(human_paths);
    for idx in range(dot_product.shape[0]):
        dot_sorted=dot_product[idx,idx_sort[idx]];   
        horse_curr=horse_paths[idx];

        if top_num>0:
            human_best=[human_paths[idx_curr] for idx_curr in idx_sort[idx,:top_num]];
            best_row=[horse_curr]+human_best;
            best_row=[util.getRelPath(file_curr,dir_server) for file_curr in best_row]
            img_paths.append(best_row);
            
        if bottom_num>0:
            human_worst=[human_paths[idx_curr] for idx_curr in idx_sort[idx,-bottom_num:]];
            worst_row=[horse_curr]+human_worst;
            worst_row=[util.getRelPath(file_curr,dir_server) for file_curr in worst_row]
            img_paths.append(worst_row);
    return img_paths;

def main():
    print 'hello';
    dir_server='/home/SSD3/maheen-data';
    click_str='http://vision1.idav.ucdavis.edu:1000'
    batchSize=128;
#     num_clusters=32;
    dim_size=4096;
    limit=-1;
    out_dir='/home/SSD3/maheen-data/horse_project/sanityCheckHorse_cvpr'
    out_file_dot=os.path.join(out_dir,'dot_product.npy')
    out_file_html=os.path.join(out_dir,'matches.html');
    top_num=2;
    bottom_num=0;
    
    horse_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    horse_fc6_dir=os.path.join(out_dir,'test');
    horse_numBatches=2;
    

    human_data_path='/home/SSD3/maheen-data/horse_project/neighbor_data/small_datasets/matches_5_3531_horse_minloss.txt';
    human_fc6_dir=os.path.join(out_dir,'train');
    human_numBatches=28;
    
#     getDotProduct(out_file_dot,batchSize,dim_size,limit,\
#                 horse_data_path,horse_numBatches,horse_fc6_dir,\
#                   human_data_path,human_numBatches,human_fc6_dir);
    
    dot_product=np.load(out_file_dot);
    data_horse=util.readLinesFromFile(horse_data_path);
    data_horse=[file_curr.split(' ')[0] for file_curr in data_horse]
    data_human=util.readLinesFromFile(human_data_path);
    data_human=[file_curr.split(' ')[0] for file_curr in data_human]
    
    img_paths=getImgPaths(dot_product,top_num,bottom_num,data_horse,data_human,dir_server);
    captions=[];

    for r in range(len(img_paths)):
        caption_row=[];
        for c in range(len(img_paths[r])):
            img_path_curr=img_paths[r][c];
            img_path_curr=img_path_curr.split('/')[-2:];
            img_path_curr='/'.join(img_path_curr);
            caption_row.append(img_path_curr);
        captions.append(caption_row);
    
    visualize.writeHTML(out_file_html,img_paths,captions,100,100);
    print out_file_html.replace(dir_server,click_str)
    
    print 'done'

    
def problemRemoval():
    test_data=['_04_Aug16_png/horse+head181.jpg',
    'error_deleted/horse+head122_2.jpg',
    'flickr-horses/4375.jpg',
    'flickr-horses/452.jpg',
    'flickr-horses/7397.jpg',
    'google_horse_face/horse+face25.jpg']    
    
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    val_data_path_human='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss_clean.txt';
    val_data_path_human_noIm='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss_noIm_clean.txt';
    out_file_post='_noDuplicates.txt';
    
    lines_val=util.readLinesFromFile(val_data_path);
    idx_keep=[];
    for idx_curr, line_curr in enumerate(lines_val):
        keep=True;
        for check_line in test_data:
            if check_line in line_curr:
                keep=False;
                break;
        if keep:
            idx_keep.append(idx_curr);
    
    print len(idx_keep);
    for file_curr in [val_data_path,val_data_path_human,val_data_path_human_noIm]:
        lines_curr=util.readLinesFromFile(file_curr);
        out_file_curr=file_curr[:file_curr.rindex('.')]+out_file_post;
        
        lines_keep=np.array(lines_curr)[idx_keep];
        print file_curr,lines_keep.shape,len(lines_curr),out_file_curr;
        util.writeFile(out_file_curr,lines_keep);
    
    
problemRemoval();

186
/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt (186,) 192 /home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean_noDuplicates.txt
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss_clean.txt (186,) 192 /home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss_clean_noDuplicates.txt
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss_noIm_clean.txt (186,) 192 /home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_val_allKP_minLoss_noIm_clean_noDuplicates.txt
